In [2]:
!cd paylesshealth/ && dolt sql -q "select * from hospitals where cdm_indirect_url is not null;" -r csv > all_nulls.csv

In [52]:
import pandas as pd
# df = pd.read_csv('paylesshealth/hospitals.csv')
df = pd.read_csv('./paylesshealth-1/all_nulls.csv')

df2 = pd.read_csv('finished_possibles.csv')
ccn_list = df2["ccn"].tolist()

df = df[~df['ccn'].isin(ccn_list)]
# df = df.head(100)

In [38]:
import aiohttp
import asyncio
import re
import cgi
from pathlib import Path
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import traceback as tb

In [39]:
def create_headers(url):
    headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Host": urlparse(url).netloc,
    "Referer": urlparse(url).netloc
    }
    return headers

In [4]:
async def checker(session, url):
    find_strings = ["price transparency", "standard charges", "pricing", "billing", "prices", "price listing", "charges", "transparency", "standard services"]
    url_string = ["price-transparency", "standard-charges", "pricing", "billing", "prices", "price-list", "price-listing", "charges", "transparency", "standard-services"]
    if pd.isna(url):
        return pd.NA, pd.NA
    
    # if "/../" in url or "|" in url: # aiohttp is kinda not smart
    #     return False

    headers = create_headers(url)
    
    try:
        async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
            if r:
                r = await r.text()
                soup = BeautifulSoup(r, "html.parser")
                
                r = r.lower()
                
                # Check for common price transparency names
                if any(string in r for string in find_strings):
                    matching_str = [string for string in find_strings if string in r]

                    possible_indirects = []
                    for t_url in soup.find_all('a'):
                        t_url = t_url.get("href")
                        
                        if t_url:
                            if any(string in t_url for string in url_string):
                                if t_url not in possible_indirects:
                                    possible_indirects.append(t_url)

                    if not possible_indirects:
                        # possible_indirects = pd.NA
                        return pd.NA, pd.NA
                    else:
                        possible_indirects = " | ".join(possible_indirects)
                    
                    return possible_indirects, matching_str
                
                else:
                    return pd.NA, pd.NA
                    
    except aiohttp.ClientResponseError as e:
#         print("ERROR", e, url)
        return pd.NA, pd.NA

    except asyncio.TimeoutError:
        return pd.NA, pd.NA

    except aiohttp.ClientConnectionError:
        return pd.NA, pd.NA    
    
    except KeyboardInterrupt:
        import os; os.exit()

    # except UnicodeDecodeError:
    #     tb.print_exc()        
    except Exception as e:
        tb.print_exc()
        print(e)
        return pd.NA, pd.NA

In [45]:
from tqdm.asyncio import tqdm

async def main():
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df[["possible_indirect", "found_string"]] = await tqdm.gather(*(checker(session, url) for url in df["homepage"]), leave=True, position=0)
        return df

In [6]:
t = await main()

100%|██████████| 26/26 [00:04<00:00,  6.42it/s]
c:\Users\adria\miniconda3\envs\scraper\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [7]:
t = await main()

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url,possible_indirect,found_string
0,054155,"BAKERSFIELD BEHAVIORAL HEALTHCARE HOSPITAL, LLC",CA,BAKERSFIELD,https://www.bakersfieldbehavioral.com/,NaN,https://www.bakersfieldbehavioral.com/sites/de...,<NA>,<NA>
1,061319,ST VINCENT GENERAL HOSPITAL DISTRICT,CO,LEADVILLE,https://svghd.org/,NaN,https://www.svghd.org/wp-content/uploads/2019/...,<NA>,[transparency]
2,180067,UNIVERSITY OF KENTUCKY HOSPITAL,KY,LEXINGTON,https://ukhealthcare.uky.edu/,NaN,https://cleverleypteusstatic.blob.core.windows...,<NA>,"[billing, transparency]"
3,184004,EASTERN STATE HOSPITAL,KY,LEXINGTON,https://ukhealthcare.uky.edu/,NaN,https://cleverleypteusstatic.blob.core.windows...,<NA>,"[billing, transparency]"
4,189802,UNIVERSITY OF KENTUCKY CHANDLER HOSPITAL,KY,LEXINGTON,https://ukhealthcare.uky.edu/,NaN,https://cleverleypteusstatic.blob.core.windows...,<NA>,"[billing, transparency]"
5,190111,"WILLIS KNIGHTON MEDICAL CENTER, INC",LA,SHREVEPORT,https://www.wkhs.com/,NaN,https://cleverleypteusstatic.blob.core.windows...,[/patients-and-visitors/information-for-patien...,[billing]
6,031301,BENSON HOSPITAL,AZ,BENSON,https://crh.arizona.edu/benson-hospital,NaN,https://search.hospitalpriceindex.com/hpi2/hos...,<NA>,<NA>
7,034024,AURORA BEHAVIORAL HEALTH SYSTEM,AZ,GLENDALE,https://www.auroraarizona.com/,NaN,https://www.auroraarizona.com/sites/default/fi...,<NA>,<NA>
8,034028,AURORA BEHAVIORAL HEALTHCARE-TEMPE,AZ,TEMPE,https://www.auroraarizona.com/,NaN,https://www.auroraarizona.com/sites/default/fi...,<NA>,<NA>
9,041330,PIGGOTT COMMUNITY HOSPITAL,AR,PIGGOTT,https://piggotthealthsystem.com/,NaN,https://piggotthealthsystem.com/wp-content/upl...,<NA>,[cms]


In [9]:
t.dropna(subset=["found_string"], inplace=True)

t = t[t["homepage"].str.contains("https://www.va.gov/")==False] # Remove VA hospitals

t

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url,possible_indirect,found_string
0,241374,PIPESTONE COUNTY MEDICAL CENTER,MN,PIPESTONE,https://www.pipestonehealth.org/,NaN,https://www.avera.org/app/files/public/a8a06bc...,[https://cms7.revize.com/revize/security/index...,[cms]
1,180067,UNIVERSITY OF KENTUCKY HOSPITAL,KY,LEXINGTON,https://ukhealthcare.uky.edu/,NaN,https://cleverleypteusstatic.blob.core.windows...,<NA>,"[billing, transparency]"
2,184004,EASTERN STATE HOSPITAL,KY,LEXINGTON,https://ukhealthcare.uky.edu/,NaN,https://cleverleypteusstatic.blob.core.windows...,<NA>,"[billing, transparency]"
3,189802,UNIVERSITY OF KENTUCKY CHANDLER HOSPITAL,KY,LEXINGTON,https://ukhealthcare.uky.edu/,NaN,https://cleverleypteusstatic.blob.core.windows...,<NA>,"[billing, transparency]"
4,190111,"WILLIS KNIGHTON MEDICAL CENTER, INC",LA,SHREVEPORT,https://www.wkhs.com/,NaN,https://cleverleypteusstatic.blob.core.windows...,[/patients-and-visitors/information-for-patien...,[billing]
5,050177,VENTURA COUNTY MEDICAL CENTER - SANTA PAULA HO...,CA,SANTA PAULA,https://hospitals.vchca.org/,NaN,https://apps.para-hcfs.com/PTT/FinalLinks/Repo...,<NA>,"[price transparency, transparency]"
9,140181,SOUTH SHORE HOSPITAL,IL,CHICAGO,https://www.southshorehospital.com/,NaN,https://apps.para-hcfs.com/PTT/FinalLinks/Repo...,<NA>,"[price transparency, transparency]"
10,041330,PIGGOTT COMMUNITY HOSPITAL,AR,PIGGOTT,https://piggotthealthsystem.com/,NaN,https://piggotthealthsystem.com/wp-content/upl...,<NA>,[cms]
11,254010,MS STATE HOSPITAL-WHITFIELD,MS,WHITFIELD,https://www.msh.state.ms.us/,NaN,http://www.msh.state.ms.us/Bids_and_public_not...,[http://www.transparency.mississippi.gov/defau...,"[charges, transparency]"
12,260050,MOSAIC MEDICAL CENTER - MARYVILLE,MO,MARYVILLE,https://www.mymlc.com/,NaN,https://www.mymlc.com/contentassets/594d450587...,<NA>,[billing]


In [60]:
t.to_csv("possible_indirects_2.csv", index=False)